In [1]:
import numpy as numpy
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense,Input
from keras import optimizers
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
(train_data,train_labels), (test_data,test_labels) = mnist.load_data()

In [3]:
print("train_data.shape:{}".format(train_data.shape))
print("train_labels.shape:{}".format(train_labels.shape))
print("test_data.shape:{}".format(test_data.shape))
print("test_labels.shape:{}".format(test_labels.shape))

train_data.shape:(60000, 28, 28)
train_labels.shape:(60000,)
test_data.shape:(10000, 28, 28)
test_labels.shape:(10000,)


In [4]:
train_data = train_data.reshape(train_data.shape[0],-1)/255
test_data = test_data.reshape(test_data.shape[0],-1)/255

input_shape = (train_data.shape[-1],)

In [5]:
autoencoder = Sequential()
autoencoder.add(Dense(units=512,activation="elu",input_shape=input_shape))
autoencoder.add(Dense(units=128,activation="elu"))
autoencoder.add(Dense(units=10,activation="linear",name="bottleneck"))
autoencoder.add(Dense(units=128,activation="elu"))
autoencoder.add(Dense(units=512,activation="elu"))
autoencoder.add(Dense(units=train_data.shape[-1],activation="sigmoid"))

In [6]:
loss = "mean_squared_error"
optimizer = Adam()
# metrics = ["accuracy"]
autoencoder.compile(loss=loss,
                    optimizer=optimizer)
                    # metrics = metrics)

In [7]:
autoencoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
bottleneck (Dense)           (None, 10)                1290      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1408      
_________________________________________________________________
dense_4 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_5 (Dense)              (None, 784)               402192    
Total params: 938,522
Trainable params: 938,522
Non-trainable params: 0
________________________________________________

In [8]:
batch_size = 1024
epochs = 10
verbose = 1
validation_data = (test_data,test_data)

trained_model = autoencoder.fit(train_data,train_data,
                                batch_size=batch_size,
                                epochs=epochs,
                                verbose=verbose,
                                validation_data=validation_data)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 18us/step - loss: 0.0741 - val_loss: 0.0475
Epoch 2/10
60000/60000 [==============================] - 1s 12us/step - loss: 0.0417 - val_loss: 0.0367
Epoch 3/10
60000/60000 [==============================] - 1s 11us/step - loss: 0.0341 - val_loss: 0.0302
Epoch 4/10
60000/60000 [==============================] - 1s 12us/step - loss: 0.0286 - val_loss: 0.0263
Epoch 5/10
60000/60000 [==============================] - 1s 11us/step - loss: 0.0258 - val_loss: 0.0244
Epoch 6/10
60000/60000 [==============================] - 1s 12us/step - loss: 0.0241 - val_loss: 0.0229
Epoch 7/10
60000/60000 [==============================] - 1s 11us/step - loss: 0.0228 - val_loss: 0.0219
Epoch 8/10
60000/60000 [==============================] - 1s 11us/step - loss: 0.0218 - val_loss: 0.0210
Epoch 9/10
60000/60000 [==============================] - 1s 12us/step - loss: 0.0208 - val_loss: 0.0201
Epoch

In [9]:
encoder = Model(autoencoder.input,
                autoencoder.get_layer("bottleneck").output)
encoder_data = encoder.predict(train_data)

In [10]:
encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1_input (InputLayer)   (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
bottleneck (Dense)           (None, 10)                1290      
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________


In [14]:
decoded_output = autoencoder.predict(train_data)

In [16]:
encoding_dim = 10

encoded_input = Input(shape=(encoding_dim,))
decoder = autoencoder.layers[-3](encoded_input)
decoder = autoencoder.layers[-2](decoder)
decoder = autoencoder.layers[-1](decoder)
decoder = Model(encoded_input,decoder)

In [39]:
decoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1408      
_________________________________________________________________
dense_4 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_5 (Dense)              (None, 784)               402192    
Total params: 469,648
Trainable params: 469,648
Non-trainable params: 0
_________________________________________________________________


In [44]:
from keras.preprocessing import image